In [1]:
import nltk
from nltk.corpus import stopwords
import csv
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from tqdm import tqdm
from datetime import datetime
import matplotlib.pyplot as plt
from indexing import IndexType, BasicInvertedIndex, InvertedIndex
from rank_bm25 import BM25Okapi
import ranker
import numpy as np
from nltk.tokenize import word_tokenize
import nltk
import ast

2023-12-14 15:24:41.225961: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-14 15:24:41.226059: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-14 15:24:42.921327: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-14 15:24:45.644417: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-14 15:25:01.805212: W tensorflow/comp

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/yyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/yyxia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
filtered_df = pd.read_csv('/home/yyxia/si650_project/output_tokenized.csv')

In [5]:
filtered_df['review_text_list'] = filtered_df['review_text_list'].apply(ast.literal_eval)

In [6]:
filtered_df['titleNsum'] = filtered_df['titleNsum'].apply(ast.literal_eval)

In [7]:
bm25_model = BM25Okapi(filtered_df['review_text_list'].tolist())

In [8]:
def searchEngine(input_query:str, dataframe, doc_column_name):
    bm25_model =  BM25Okapi(dataframe[doc_column_name].tolist())
    doc_scores = bm25_model.get_scores(word_tokenize(input_query))
    idx_of_max_score = np.argmax(doc_scores)
    print('Dociument ID --', dataframe['docid'][idx_of_max_score])
    print('Product ID --', dataframe['product/productId'][idx_of_max_score])
    print('Product Name --', dataframe['product/title'][idx_of_max_score])
    print('Product Price --', dataframe['product/price'][idx_of_max_score])
    print('Product Rating Score --', dataframe['review/score'][idx_of_max_score])
    print('Product Review Helpfulness --', dataframe['review/helpfulness'][idx_of_max_score])
    print('Product Comments --', dataframe['review/text'][idx_of_max_score])
    return np.max(doc_scores)

In [11]:
# top 10
def searchEngine(input_query: str, dataframe, doc_column_name, top_n=50):
    bm25_model = BM25Okapi(dataframe[doc_column_name].tolist())
    doc_scores = bm25_model.get_scores(word_tokenize(input_query))
    
    # Get indices of top N documents
    top_n_indices = np.argsort(doc_scores)[::-1][:top_n]
    
    # Print information for the top N documents
    for idx in top_n_indices:
        print(f'\nResult {top_n - top_n_indices.tolist().index(idx)}:')
        print('Document ID:', dataframe['docid'][idx])
        print('Product ID:', dataframe['product/productId'][idx])
        print('Product Name:', dataframe['product/title'][idx])
        print('Product Price:', dataframe['product/price'][idx])
        print('Product Rating Score:', dataframe['review/score'][idx])
        print('Product Review Helpfulness:', dataframe['review/helpfulness'][idx])
        print('Product Comments:', dataframe['review/text'][idx])
        print('score', doc_scores[idx])
    
    return np.max(doc_scores)

In [12]:
searchEngine('comfortable clothes', filtered_df, 'review_text_list')


Result 50:
Document ID: 44426
Product ID:  B000PLRFAC
Product Name:  Charlie's Soap Laundry Liquid - 1 Gal Jug - (128 Loads)
Product Price:  34.99
Product Rating Score: 5.0
Product Review Helpfulness:  1/1
Product Comments: ['use', 'many', 'times', 'wash', 'baby', 'clothes', 'stuffs', 'liquid', 'required', 'time', 'feels', 'comfortable', 'strongly', 'recommend', 'item']
score 11.774878633048488

Result 49:
Document ID: 99281
Product ID:  B000GHL62I
Product Name:  Wellgate for Women Gel-Comfort Knee Support, 1 Size
Product Price:  18.19
Product Rating Score: 5.0
Product Review Helpfulness:  2/2
Product Comments: ['4th', 'knee', 'support', 'tried', 'wish', 'bought', 'first', 'comfortable', 'supportive', 'bulky', 'clothes', 'silicone', 'around', 'kneecap', 'gets', 'bit', 'uncomfortable', 'end', 'day', 'wear', 'good', 'portion', 'day', 'without', 'bothering', 'best', 'one', 'yet']
score 9.927110098892978

Result 48:
Document ID: 44739
Product ID:  B000KJJX0Y
Product Name:  Sierra Comfort 

11.774878633048488

In [14]:
searchEngine('travel socks', filtered_df, 'review_text_list')


Result 50:
Document ID: 12539
Product ID:  B0000ZSJ94
Product Name:  Activa Mens Dress Socks Moderate Compression 20-30 mmHg
Product Price:  8.95
Product Rating Score: 4.0
Product Review Helpfulness:  2/2
Product Comments: ['like', 'wear', 'support', 'socks', 'times', 'air', 'travel', 'comfortable', 'reasonably', 'priced']
score 14.108976326582637

Result 49:
Document ID: 49652
Product ID:  B00011F0LW
Product Name:  Activa Mens Dress Socks Moderate Compression 20-30 mmHg
Product Price:  12.81
Product Rating Score: 4.0
Product Review Helpfulness:  2/2
Product Comments: ['like', 'wear', 'support', 'socks', 'times', 'air', 'travel', 'comfortable', 'reasonably', 'priced']
score 14.108976326582637

Result 48:
Document ID: 11877
Product ID:  B0000ZSJ62
Product Name:  Activa Mens Dress Socks Moderate Compression 20-30 mmHg
Product Price:  12.10
Product Rating Score: 4.0
Product Review Helpfulness:  2/2
Product Comments: ['like', 'wear', 'support', 'socks', 'times', 'air', 'travel', 'comforta

14.108976326582637

In [15]:
filtered_df.head()

,product/productId,product/title,product/price,review/userId,review/profileName,review/helpfulness,review/score,review/time,review/summary,review/text,time,docid,review_text_list,titleNsum,combined_text
0,B000278ADA,Jobst Ultrasheer 15-20 Knee-High Silky Beige ...,46.34,ASL7LZAL2LBMS,"Ohio Nurse ""bronwyn59""",13/13,5.0,1157241600,Lower leg comfort for 12 hours +,"['excellent', 'stockings', 'long', 'shifts', '...",2006,1,"[excellent, stockings, long, shifts, feet, tig...","[jobst, ultrasheer, 15, 20, knee, high, silky,...","['jobst', 'ultrasheer', '15', '20', 'knee', 'h..."
1,B000278ADA,Jobst Ultrasheer 15-20 Knee-High Silky Beige ...,46.34,A21F8C1MDLQDTH,S. Lawson,14/15,5.0,1110326400,Fashionable Compression Stockings!,"['dvt', 'doctor', 'required', 'wear', 'compres...",2005,2,"[dvt, doctor, required, wear, compression, sto...","[jobst, ultrasheer, 15, 20, knee, high, silky,...","['jobst', 'ultrasheer', '15', '20', 'knee', 'h..."
2,B000278ADA,Jobst Ultrasheer 15-20 Knee-High Silky Beige ...,46.34,A3J4K29TNC7RV1,"Karen W. ""Karen W.""",13/14,5.0,1173312000,Sheer Relief!,"['doctor', 'recommended', 'thigh', 'high', 've...",2007,3,"[doctor, recommended, thigh, high, version, pr...","[jobst, ultrasheer, 15, 20, knee, high, silky,...","['jobst', 'ultrasheer', '15', '20', 'knee', 'h..."
3,B000278ADA,Jobst Ultrasheer 15-20 Knee-High Silky Beige ...,46.34,A3GQAVM66FA88J,"G. Alosa ""wifey""",6/6,5.0,1204416000,No more uncomfortable flights,"['stockings', 'fit', 'well', 'comfortable', 'o...",2008,4,"[stockings, fit, well, comfortable, opportunit...","[jobst, ultrasheer, 15, 20, knee, high, silky,...","['jobst', 'ultrasheer', '15', '20', 'knee', 'h..."
4,B000278ADA,Jobst Ultrasheer 15-20 Knee-High Silky Beige ...,46.34,A1BTRWL1E3T2YB,A. S. Penuel,6/6,5.0,1184803200,Jobst UltraSheer Thigh High,"['excellent', 'product', 'however', 'difficult...",2007,5,"[excellent, product, however, difficult, get, ...","[jobst, ultrasheer, 15, 20, knee, high, silky,...","['jobst', 'ultrasheer', '15', '20', 'knee', 'h..."


In [74]:
group_df = filtered_df[['product/title', 'review_text_list']]
group_df['productId'] = group_df.index + 1
group_df.head()

/tmp/ipykernel_4162382/4003140671.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group_df['productId'] = group_df.index + 1


,product/title,review_text_list,productId
0,Jobst Ultrasheer 15-20 Knee-High Silky Beige ...,"[excellent, stockings, long, shifts, feet, tig...",1
1,Jobst Ultrasheer 15-20 Knee-High Silky Beige ...,"[dvt, doctor, required, wear, compression, sto...",2
2,Jobst Ultrasheer 15-20 Knee-High Silky Beige ...,"[doctor, recommended, thigh, high, version, pr...",3
3,Jobst Ultrasheer 15-20 Knee-High Silky Beige ...,"[stockings, fit, well, comfortable, opportunit...",4
4,Jobst Ultrasheer 15-20 Knee-High Silky Beige ...,"[excellent, product, however, difficult, get, ...",5


In [75]:
grouped_df = group_df.groupby('product/title')['review_text_list'].agg(sum).reset_index()
grouped_df['productId'] = grouped_df.index + 1
grouped_df.head()

,product/title,review_text_list,productId
0,#1 Rated Diaper Happy Heiny's Pocket Diaper w...,"[deliver, promised, easy, use, wash, leaking, ...",1
1,(Pack of 3) Reach Access Power Flosser (Color...,"[cannot, help, assert, reach, access, daily, f...",2
2,1 Step-Compac 14102 1 Step Tub Shower Cleaner...,"[love, little, pads, work, well, shower, alway...",3
3,100 Durex Maximum Love Condoms NEW! Larger an...,"[purchased, company, green, web, properties, c...",4
4,100 Wilkinson Sword Classic Double Edge Safet...,"[order, went, flawlessly, delivered, within, t...",5


In [76]:
grouped_df.to_csv('grouped_df.csv', index=False)

In [19]:
grouped_df = pd.read_csv('grouped_df.csv')

In [20]:
grouped_df

,product/title,review_text_list,productId
0,#1 Rated Diaper Happy Heiny's Pocket Diaper w...,"['deliver', 'promised', 'easy', 'use', 'wash',...",1
1,(Pack of 3) Reach Access Power Flosser (Color...,"['cannot', 'help', 'assert', 'reach', 'access'...",2
2,1 Step-Compac 14102 1 Step Tub Shower Cleaner...,"['love', 'little', 'pads', 'work', 'well', 'sh...",3
3,100 Durex Maximum Love Condoms NEW! Larger an...,"['purchased', 'company', 'green', 'web', 'prop...",4
4,100 Wilkinson Sword Classic Double Edge Safet...,"['order', 'went', 'flawlessly', 'delivered', '...",5
...,...,...,...
3225,iRobot 5950 Clorox Scooba Cleaning Solution,"['product', 'works', 'well', 'iscooba', 'meant...",3226
3226,iSatori MX-LS7-Contain 7-Keto and Coleus Fors...,"['product', 'one', 'best', 'potentially', 'bes...",3227
3227,iWALKFREE Rehab Rehabilitation Knee Walker Cr...,"['buy', 'amazon', 'feel', 'compelled', 'contri...",3228
3228,"intraMAX Liquid Nutrition, Peach Mango Flavor...","['title', 'sounds', 'bit', 'dramatic', 'taking...",3229


In [22]:
import ast
grouped_df['review_text_list'] = grouped_df['review_text_list'].apply(ast.literal_eval)

In [96]:
def searchEngine(input_query: str, dataframe, doc_column_name, result_df, top_n=50):
    bm25_model = BM25Okapi(dataframe[doc_column_name].tolist())
    doc_scores = bm25_model.get_scores(word_tokenize(input_query))
    
    # Get indices of top N documents
    top_n_indices = np.argsort(doc_scores)[::-1][:top_n]
    
    # Print information for the top N documents
    for idx in top_n_indices:
        print(f'\nResult {top_n_indices.tolist().index(idx) + 1}:')
        
        print('Product Title: ', dataframe['product/title'][idx])
        print('Product Id: ', dataframe['productId'][idx])
        print('score: ', doc_scores[idx])
        result_df = result_df.append({
        'result_index': top_n_indices.tolist().index(idx)+1,
        'product_title': dataframe['product/title'][idx],
        'product_id': dataframe['productId'][idx],
        'score': doc_scores[idx],
        'query': input_query
    }, ignore_index=True)
    return result_df
#     return np.max(doc_scores)

In [97]:
result_df = pd.DataFrame(columns=['result_index', 'product_title', 'product_id', 'score'])

In [98]:
result_df = searchEngine('comfortable clothes', grouped_df, 'review_text_list', result_df)


Result 1:
Product Title:   Lansinoh 20265 Disposable Nursing Pads, 60-Count Boxes (Pack of 3)
Product Id:  1446
score:  6.708870068021579

Result 2:
Product Title:   Seventh Generation Natural Laundry Detergent, Baby, Ultra-Concentrated Liquid, 50-Ounces Bottles (Pack of 6)
Product Id:  2622
score:  6.559673310077099

Result 3:
Product Title:   Charlie's Soap Laundry Liquid - 1 Gal Jug - (128 Loads)
Product Id:  503
score:  6.4246625151826775

Result 4:
Product Title:   Household Essentials Hanging Cotton Canvas Laundry Hamper Bag
Product Id:  1270
score:  6.132265235706047

Result 5:
Product Title:   Wellgate for Women Gel-Comfort Knee Support, 1 Size
Product Id:  3151
score:  6.1318721089573005

Result 6:
Product Title:   All Small & Mighty, Triple Concentrated Liquid Laundry Detergent, Nine 32-Load Bottles (288-Loads)
Product Id:  99
score:  6.116672602464715

Result 7:
Product Title:   Lansinoh Disposable Nursing Pads
Product Id:  1447
score:  6.093843818803553

Result 8:
Product 

/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [100]:
result_df

,result_index,product_title,product_id,score,query
0,1,"Lansinoh 20265 Disposable Nursing Pads, 60-Co...",1446,6.708870,comfortable clothes
1,2,"Seventh Generation Natural Laundry Detergent,...",2622,6.559673,comfortable clothes
2,3,Charlie's Soap Laundry Liquid - 1 Gal Jug - (...,503,6.424663,comfortable clothes
3,4,Household Essentials Hanging Cotton Canvas La...,1270,6.132265,comfortable clothes
4,5,"Wellgate for Women Gel-Comfort Knee Support, ...",3151,6.131872,comfortable clothes
5,6,"All Small & Mighty, Triple Concentrated Liqui...",99,6.116673,comfortable clothes
6,7,Lansinoh Disposable Nursing Pads,1447,6.093844,comfortable clothes
7,8,Cramer E6 Groin Hip Spica,627,6.062437,comfortable clothes
8,9,"Duro-Med Posture Corrector, White,",837,5.937813,comfortable clothes
9,10,OTC Elastic Abdominal Binder,2058,5.873750,comfortable clothes


In [101]:
result_df = searchEngine('warm clothes', grouped_df, 'review_text_list', result_df)


Result 1:
Product Title:   Thermoskin Lumbar Support in Beige
Product Id:  2949
score:  9.11019539769403

Result 2:
Product Title:   Mrs. Meyer's Clean Day Laundry Liquid Detergent, Lemon Verbena, 64-Ounce Bottles (Case of 6)
Product Id:  1722
score:  8.98369429601904

Result 3:
Product Title:   Seventh Generation Laundry Detergent, White Flower & Bergamot Citrus, Powder, 112-Ounce Boxes (Pack of 4)
Product Id:  2618
score:  8.069864833495744

Result 4:
Product Title:   Biokleen Premium Plus Laundry Powder, 5-Lb. Boxes (Pack of 8)
Product Id:  295
score:  7.897961504278609

Result 5:
Product Title:   Tide HE Detergent for High Efficiency Washers, Powder, Original Scent, 31-Load Boxes (Pack of 4)
Product Id:  2961
score:  7.795492556916674

Result 6:
Product Title:   Seventh Generation Natural Laundry Detergent, Lavender Scented, Ultra-Concentrated Liquid, 100-Ounce Bottles (Pack of 4)
Product Id:  2624
score:  7.586549938753785

Result 7:
Product Title:   Duro-Med Mealtime Protector
P

/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [103]:
result_df = searchEngine('travel', grouped_df, 'review_text_list', result_df)


Result 1:
Product Title:   Marpac TSC 330 Travel Sound Conditioner
Product Id:  1580
score:  3.447555744217515

Result 2:
Product Title:   Dex Products Travel Wipes Warmer
Product Id:  715
score:  3.441064531467894

Result 3:
Product Title:   MB9910T Slim Line Reader in Metal Tube with Spring Hinge and Clip
Product Id:  1540
score:  3.4322607292805394

Result 4:
Product Title:   Chrome METAL SHAVING SOAP BOWL silver shave dish with LID covered holder NEW Great for Travel
Product Id:  527
score:  3.4142681678018945

Result 5:
Product Title:   Activa Mens Dress Socks Moderate Compression 20-30 mmHg
Product Id:  76
score:  3.381810679341252

Result 6:
Product Title:   The Stick Travel Stick
Product Id:  2924
score:  3.3251862304182733

Result 7:
Product Title:   Braun PocketGo P-70 Men's Shaver
Product Id:  374
score:  3.3163080385517745

Result 8:
Product Title:   Floxite 5X Lighted Magnifying Mirror 5" white
Product Id:  1007
score:  3.3068088396150466

Result 9:
Product Title:   Medpo

/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [104]:
result_df = searchEngine('conditioner', grouped_df, 'review_text_list', result_df)


Result 1:
Product Title:   Avalon Organics Conditioner Strenthening Peppermint -- 11 fl oz
Product Id:  185
score:  8.960816958013497

Result 2:
Product Title:   Avalon Organic Conditioner
Product Id:  183
score:  8.960816958013497

Result 3:
Product Title:   Avalon Organics Conditioner Awapuhi Mango Moisturizing
Product Id:  184
score:  8.960816958013497

Result 4:
Product Title:   Giovanni Cosmetics Conditioner Smooth As Silk
Product Id:  1097
score:  8.816515309981531

Result 5:
Product Title:   Thin to Thick Extra Volume Shampoo
Product Id:  2953
score:  8.558560024643384

Result 6:
Product Title:   Avalon Shampoo
Product Id:  186
score:  8.471715278335655

Result 7:
Product Title:   TheraNeem Gentle Therape Shampoo - 12 oz - Liquid
Product Id:  2932
score:  8.467248365606803

Result 8:
Product Title:   RPS 16-Ounce Waterbed Conditioner
Product Id:  2461
score:  8.356940084388999

Result 9:
Product Title:   Lily Of The Desert Aloe Vera Juice
Product Id:  1493
score:  7.94426632245

/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [105]:
result_df = searchEngine('organic', grouped_df, 'review_text_list', result_df)


Result 1:
Product Title:   Ginseng Vitality Tea 16 Bags
Product Id:  1096
score:  5.536966116372739

Result 2:
Product Title:   Active Green Bar
Product Id:  78
score:  5.495690618279103

Result 3:
Product Title:   Natracare Organic All Cotton Tampons, Regular with Applicator, 16 Count boxes (Pack of 12)
Product Id:  1792
score:  5.284243145917317

Result 4:
Product Title:   Dynamic Health Laboratories Organic Apple Cider Vinegar with Mother
Product Id:  853
score:  5.258836742081907

Result 5:
Product Title:   Oskri Organics Coconut Bar, Original, 1.86 Ounce Bars - (Pack of 18)
Product Id:  2178
score:  5.172013669962369

Result 6:
Product Title:   Dr. Bronner'S Organic Bar Soaps Pure Castile
Product Id:  764
score:  5.1696201910273745

Result 7:
Product Title:   Hibiscus Tea - Certified Organic - 24 Teabags
Product Id:  1219
score:  5.141452362642693

Result 8:
Product Title:   New Chapter Every Woman Multivitamins
Product Id:  1934
score:  5.083232558188446

Result 9:
Product Title

/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [106]:
result_df = searchEngine('healthy', grouped_df, 'review_text_list', result_df)


Result 1:
Product Title:   Active Green Bar
Product Id:  78
score:  2.8240335552519804

Result 2:
Product Title:   Healthy Trinity (Dairy-free) by Natren - 60 Capsules Probiotics
Product Id:  1181
score:  2.8187637541340322

Result 3:
Product Title:   Twinlab Cod Liver Oil Sg
Product Id:  3016
score:  2.7900905077026406

Result 4:
Product Title:   Larabar Fruit and Nut Food Bar, 1.7-Ounce Bars (Pack of 16)
Product Id:  1450
score:  2.779060976964611

Result 5:
Product Title:   Kyolic Kyo-Dophilus
Product Id:  1423
score:  2.763060570338477

Result 6:
Product Title:   New Chapter Every Woman Multivitamins
Product Id:  1934
score:  2.7167272554964756

Result 7:
Product Title:   Fruit and Veggie Bowl with Removable Ice Pack
Product Id:  1024
score:  2.706045616452807

Result 8:
Product Title:   O'Keefes Creme
Product Id:  2048
score:  2.6835374451397533

Result 9:
Product Title:   Hibiscus Tea - Certified Organic - 24 Teabags
Product Id:  1219
score:  2.671421291942929

Result 10:
Produc

/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [107]:
result_df = searchEngine('weight control', grouped_df, 'review_text_list', result_df)


Result 1:
Product Title:   Lipovarin 150 Caps Thermogenic Weight Loss Diet Pill
Product Id:  1497
score:  4.522241946056331

Result 2:
Product Title:   Moist Heating Pads - TheraTherm Digital Moist Heating Pads
Product Id:  1696
score:  4.409608980421396

Result 3:
Product Title:   EAS AdvantEdge Carb Control Nutrition Bar
Product Id:  858
score:  4.329567924644511

Result 4:
Product Title:   Weight Loss with Hypnosis CD
Product Id:  3139
score:  4.292120041293205

Result 5:
Product Title:   Stacker 2 Ephedra Free
Product Id:  2797
score:  4.278302973338654

Result 6:
Product Title:   Thyrotril 90 Caps Metabolism Thyroid Sterling Grant
Product Id:  2960
score:  4.191423845144166

Result 7:
Product Title:   Lipovarin - Sterling Grant 90 Caps
Product Id:  1496
score:  4.189130399259684

Result 8:
Product Title:   SlimFast High Protein Shake Mix, 12.83 Ounce (Pack of 3)
Product Id:  2659
score:  4.185978209945115

Result 9:
Product Title:   Bojenmi Chinese Diet Tea to Support Weight Cont

/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [108]:
result_df = searchEngine('nutrition', grouped_df, 'review_text_list', result_df)


Result 1:
Product Title:   PowerBar Harvest Whole Grain Energy Bar, 2.29-Ounce Bars (Pack of 15)
Product Id:  2366
score:  5.664126135872037

Result 2:
Product Title:   Ensure Complete Balanced Nutrition Drink, Butter Pecan, 8-Ounce (Pack of 24)
Product Id:  918
score:  5.582863742363884

Result 3:
Product Title:   PureFit Nutrition Bar, 2-Ounce Bars (Pack of 15)
Product Id:  2447
score:  5.581511487401936

Result 4:
Product Title:   CytoSport Muscle Milk Ready-to-Drink Shake
Product Id:  665
score:  5.561462410923644

Result 5:
Product Title:   Champion Nutrition Pure Whey Protein Stack, 60 packet box
Product Id:  499
score:  5.4628864587853245

Result 6:
Product Title:   Optimum Nutrition Glutamine Powder
Product Id:  2136
score:  5.413836879288304

Result 7:
Product Title:   Clif Bar - 12 Pack
Product Id:  552
score:  5.399727162943041

Result 8:
Product Title:   Complete Whole Food Health Optimizer Natural Flavor By Vega - 36.6 Ounces
Product Id:  596
score:  5.344072435486119

Re

/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [109]:
result_df = searchEngine('engergy bar', grouped_df, 'review_text_list', result_df)


Result 1:
Product Title:   GURU Energy Drink, 100% Natural, 8.4-Fluid Ounce Can (Pack of 24)
Product Id:  1049
score:  11.154833012951936

Result 2:
Product Title:   Clif Shot Bloks Organic Energy Chews, 2.1-Ounce Packets
Product Id:  556
score:  7.248028940763198

Result 3:
Product Title:   PowerBar Performance The Original Energy Bar, 2.29-Ounce Bars
Product Id:  2367
score:  5.768600141135286

Result 4:
Product Title:   Active Green Bar
Product Id:  78
score:  5.768444467947522

Result 5:
Product Title:   Promax 12 per pack, 2.64 oz. each
Product Id:  2437
score:  5.735997838264145

Result 6:
Product Title:   Detour Deluxe Whey Protein Energy Bar
Product Id:  711
score:  5.733225727657044

Result 7:
Product Title:   Medline Grab Bar
Product Id:  1629
score:  5.727792225995706

Result 8:
Product Title:   Met-Rx Big 100 Colossal Meal Replacement Bar, Super Cookie Crunch, 12 Bars, 3.52 Ounces
Product Id:  1662
score:  5.721137149965843

Result 9:
Product Title:   Clif Bar - 12 Pack
Pr

/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [110]:
result_df

,result_index,product_title,product_id,score,query
0,1,"Lansinoh 20265 Disposable Nursing Pads, 60-Co...",1446,6.708870,comfortable clothes
1,2,"Seventh Generation Natural Laundry Detergent,...",2622,6.559673,comfortable clothes
2,3,Charlie's Soap Laundry Liquid - 1 Gal Jug - (...,503,6.424663,comfortable clothes
3,4,Household Essentials Hanging Cotton Canvas La...,1270,6.132265,comfortable clothes
4,5,"Wellgate for Women Gel-Comfort Knee Support, ...",3151,6.131872,comfortable clothes
...,...,...,...,...,...
445,46,"Kashi-Go Lean Crunchy, 24 ct Variety Pack",1396,5.287611,engergy bar
446,47,"Clif Bar Energy Bar, Variety Pack of Crunchy ...",554,5.281318,engergy bar
447,48,Ka-Bar 2-1282-6 D2 Extreme Fighting Knife,1392,5.221487,engergy bar
448,49,"Clearly Natural Glycerine Soap, 4 oz - PARENT",544,5.209070,engergy bar


In [111]:
result_df = searchEngine('Best vitamins for immune system', grouped_df, 'review_text_list', result_df)


Result 1:
Product Title:   Nature's Way Pau D'Arco
Product Id:  1897
score:  11.936587122934272

Result 2:
Product Title:   Wellness Formula by Source Naturals 90 Tablets
Product Id:  3153
score:  11.431104722284685

Result 3:
Product Title:   Nature's Way SystemWell Immune System
Product Id:  1907
score:  11.047821270934506

Result 4:
Product Title:   New Chapter, Supercritical Stress Advantage - 60 softgels
Product Id:  1938
score:  11.021421628432686

Result 5:
Product Title:   Garden of Life Primal Defense
Product Id:  1056
score:  10.934634931028409

Result 6:
Product Title:   Women's Nutritional by Rainbow Light - 180 Tablets
Product Id:  3164
score:  10.924881242466311

Result 7:
Product Title:   Quantum Super Lysine Plus+
Product Id:  2453
score:  10.879652576714939

Result 8:
Product Title:   Original Formula by All One
Product Id:  2170
score:  10.716469292185069

Result 9:
Product Title:   Natural Sources ELDERBERRY CONCENTRATE
Product Id:  1821
score:  10.713032116317413



/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [112]:
result_df = searchEngine('Natural remedies for sleep', grouped_df, 'review_text_list', result_df)


Result 1:
Product Title:   Little Remedies - New Parents Survival Kit
Product Id:  1507
score:  7.871736923610273

Result 2:
Product Title:   Diphenhydramine HCI 50 Mg Allergy Medicine and Antihistamine 1000 caps
Product Id:  726
score:  7.719218183967623

Result 3:
Product Title:   Valerian Root 530 mg by Nature's Way 100 Capsules
Product Id:  3061
score:  7.6204170532285325

Result 4:
Product Title:   Passion Flower 350 mg By Solaray - 100 Capsules
Product Id:  2264
score:  7.593485980773509

Result 5:
Product Title:   Rescue Remedy (20ml vial)
Product Id:  2527
score:  7.54393790486956

Result 6:
Product Title:   Hylands Kids Kit
Product Id:  1288
score:  7.304816400874676

Result 7:
Product Title:   Lydia Pinkham Lydia Pinkham Herbal Liquid Supplement
Product Id:  1528
score:  7.287733060564939

Result 8:
Product Title:   Bach Flower Essences Rescue Sleep
Product Id:  225
score:  7.276255412515798

Result 9:
Product Title:   Ortho Molecular Product Natural D-Hist -- 120 Capsules
P

/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [113]:
result_df = searchEngine('Top-rated fitness trackers', grouped_df, 'review_text_list', result_df)


Result 1:
Product Title:   Source Naturals SAMe, 200mg
Product Id:  2755
score:  7.721283740204638

Result 2:
Product Title:   Mark of Fitness WS-820 Automatic Wrist Blood Pressure Monitor
Product Id:  1576
score:  7.506688023257014

Result 3:
Product Title:   Mark of Fitness MF-81 Wrist Blood Pressure Monitor
Product Id:  1573
score:  7.210994115921391

Result 4:
Product Title:   4Ever Fit Fruit Blast the Isolate 2 lb (908 g)
Product Id:  42
score:  6.647189352548703

Result 5:
Product Title:   Natrol Tonalin CLA 1200mg
Product Id:  1803
score:  6.62449562964389

Result 6:
Product Title:   Accu-Measure Fitness 3000 Personal Body Fat Tester
Product Id:  71
score:  6.581320777272905

Result 7:
Product Title:   Omron HBF-306C Fat Loss Monitor, Black
Product Id:  2100
score:  6.544944710736083

Result 8:
Product Title:   One-Pound Fat Replica 1Lb Fat Model Replica
Product Id:  2121
score:  6.47186871728107

Result 9:
Product Title:   Stacker 2 Ephedra Free
Product Id:  2797
score:  6.449

/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [114]:
result_df = searchEngine('Organic protein powders', grouped_df, 'review_text_list', result_df)


Result 1:
Product Title:   Bioplex Nutrition Pure WPI Nutritional Shake
Product Id:  296
score:  12.61929778989811

Result 2:
Product Title:   Dymatize Nutrition Elite Whey Protein Powder
Product Id:  850
score:  12.589808143241207

Result 3:
Product Title:   Optimum Nutrition 100% Egg Protein
Product Id:  2129
score:  12.546846068243166

Result 4:
Product Title:   Champion Nutrition Pure Whey Protein Stack, 60 packet box
Product Id:  499
score:  12.522034628127216

Result 5:
Product Title:   Jay Robb Enterprises - Whey Vanilla 24oz, 24 oz powder
Product Id:  1357
score:  12.5189914813424

Result 6:
Product Title:   BSN Syntha-6 Protein Powder
Product Id:  219
score:  12.485630935631145

Result 7:
Product Title:   Jay Robb Egg White Protein Strawberry 24oz
Product Id:  1356
score:  12.415405894250714

Result 8:
Product Title:   Champion Nutrition WHEY STACK BANANA 2.2LB
Product Id:  501
score:  12.339812764843554

Result 9:
Product Title:   Designer Whey - Designer Whey Strawberry, 2 

/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [115]:
result_df = searchEngine('Effective allergy relief medications', grouped_df, 'review_text_list', result_df)


Result 1:
Product Title:   ENTSOL Spray, Buffered Hypertonic Saline Nasal Spray - 100 ml
Product Id:  861
score:  13.710653817448165

Result 2:
Product Title:   Claritin Allergy 24 Hour Tablets, 10 mg
Product Id:  529
score:  13.219622576345653

Result 3:
Product Title:   Ortho Molecular Product Natural D-Hist -- 120 Capsules
Product Id:  2175
score:  13.03935747833351

Result 4:
Product Title:   Sinucleanse Sinucleanse Neti Pot All Natural Nasal Wash System Kit
Product Id:  2649
score:  12.115732644068967

Result 5:
Product Title:   NOW QUERCETIN with BROMELAIN - 120 Vcaps
Product Id:  1781
score:  11.427490603139127

Result 6:
Product Title:   Boiron - Sabadil, 60 tablets
Product Id:  332
score:  11.308796079093375

Result 7:
Product Title:   Sinus Buster Classic Formula by Sinus Buster - .68 Ounces
Product Id:  2650
score:  11.26431296879404

Result 8:
Product Title:   Biotene Dental Products Biotene Oralbalance Dry Mouth Moisturizer Gel
Product Id:  300
score:  11.147877946741673


/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [116]:
result_df

,result_index,product_title,product_id,score,query
0,1,"Lansinoh 20265 Disposable Nursing Pads, 60-Co...",1446,6.708870,comfortable clothes
1,2,"Seventh Generation Natural Laundry Detergent,...",2622,6.559673,comfortable clothes
2,3,Charlie's Soap Laundry Liquid - 1 Gal Jug - (...,503,6.424663,comfortable clothes
3,4,Household Essentials Hanging Cotton Canvas La...,1270,6.132265,comfortable clothes
4,5,"Wellgate for Women Gel-Comfort Knee Support, ...",3151,6.131872,comfortable clothes
...,...,...,...,...,...
695,46,Ponaris Ponaris Nasal Emollient,2356,7.259691,Effective allergy relief medications
696,47,Blomdahl Silver Titanium Bezel Crystal Earrings,311,7.230895,Effective allergy relief medications
697,48,Rohto Rohto Ice Redness Relief Eye Drops,2539,7.113131,Effective allergy relief medications
698,49,Lobob Lobob Optimum Wetting And Rewetting Drops,1516,7.040209,Effective allergy relief medications


In [117]:
result_df = searchEngine('release anxiety', grouped_df, 'review_text_list', result_df)


Result 1:
Product Title:   Nature's Way B-100 Complex
Product Id:  1858
score:  10.39744207636739

Result 2:
Product Title:   Life Extension Vitamin B3 Niacin
Product Id:  1468
score:  10.035193390834309

Result 3:
Product Title:   Source Naturals Theanine Serene with Relora
Product Id:  2765
score:  9.707368956893264

Result 4:
Product Title:   Melatonin - Time Released (3mg)
Product Id:  1642
score:  9.42920979929927

Result 5:
Product Title:   Nature's Way 5-Htp 50 mg
Product Id:  1852
score:  9.356644517135

Result 6:
Product Title:   Source Naturals 5-HTP, 100mg
Product Id:  2699
score:  9.190321629931445

Result 7:
Product Title:   MindPlace Proteus USB Light & Sound Meditation Mind Machine
Product Id:  1685
score:  8.754841393352605

Result 8:
Product Title:   Stacker 3
Product Id:  2798
score:  8.657232774955474

Result 9:
Product Title:   Hylands Hylands Calms Forte Sleep Aid
Product Id:  1287
score:  8.288625583502375

Result 10:
Product Title:   Source Naturals Inositol/Cho

/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [118]:
result_df = searchEngine('Recommended probiotics for gut health', grouped_df, 'review_text_list', result_df)


Result 1:
Product Title:   Garden of Life Primal Defense Ultra Ultimate Probiotics Formula
Product Id:  1057
score:  17.261916972650052

Result 2:
Product Title:   Digestive Advantage - Daily Probiotic
Product Id:  718
score:  16.88851522397693

Result 3:
Product Title:   THORNE RESEARCH - Bacillus Coagulans (formerly Lactobacillus Sporogenes) - 60's
Product Id:  2865
score:  16.544947242225295

Result 4:
Product Title:   Garden of Life Primal Defense
Product Id:  1056
score:  15.953081433600538

Result 5:
Product Title:   Kyolic Kyo-Dophilus
Product Id:  1423
score:  15.753904144202641

Result 6:
Product Title:   Enzymatic Therapy Acidophilus Pearls
Product Id:  923
score:  15.406050562902085

Result 7:
Product Title:   Florastor - 50 count, 250mg
Product Id:  999
score:  15.237762359839348

Result 8:
Product Title:   ReNew Life Ultimate Flora Critical Care 50 Billion, Vegetable Capsules
Product Id:  2483
score:  14.936002730860336

Result 9:
Product Title:   Ortho Molecular Product 

/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [119]:
result_df = searchEngine('Essential oils for stress relief', grouped_df, 'review_text_list', result_df)


Result 1:
Product Title:   Aura Cacia - Lavender Essential Oil
Product Id:  175
score:  10.98284786152129

Result 2:
Product Title:   Source Naturals Calcium D-Glucarate 500mg
Product Id:  2708
score:  9.064915332640998

Result 3:
Product Title:   Ponaris Ponaris Nasal Emollient
Product Id:  2356
score:  8.368811623772082

Result 4:
Product Title:   Quantum Super Lysine Plus+
Product Id:  2453
score:  8.234785707134762

Result 5:
Product Title:   NOW Foods Tea Tree Oil
Product Id:  1775
score:  7.798300956696105

Result 6:
Product Title:   Irwin Naturals Women's Living Green Liquid-Gel Multi
Product Id:  1328
score:  7.765811080868085

Result 7:
Product Title:   NOW Foods Lavender Oil
Product Id:  1767
score:  7.567401012732228

Result 8:
Product Title:   Now Foods Organic Macadamia Oil Pure
Product Id:  1987
score:  7.288836509149771

Result 9:
Product Title:   Now Foods Lemon Oil
Product Id:  1981
score:  7.224545342085126

Result 10:
Product Title:   Ginseng Vitality Tea 16 Bags
Pr

/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [120]:
result_df = searchEngine('Top-rated sunscreen for sensitive skin', grouped_df, 'review_text_list', result_df)


Result 1:
Product Title:   Vanicream Lip Protectant SPF 30 - 0.35 oz
Product Id:  3062
score:  13.118156787575836

Result 2:
Product Title:   Avon SKIN-SO-SOFT Bug Guard PLUS IR3535&reg; Insect Repellent Moisturizing Lotion - Clearance SPF 30 Gentle Breeze
Product Id:  191
score:  12.613292509259649

Result 3:
Product Title:   Rit Sun Guard Laundry Treatment UV Protectant - si
Product Id:  2536
score:  12.019416420044871

Result 4:
Product Title:   Aubrey Organics
Product Id:  173
score:  11.20552459460506

Result 5:
Product Title:   NeoCutis Bio-Cream - Bio-Restorative Skin Cream with PSP
Product Id:  1924
score:  10.960820030369254

Result 6:
Product Title:   Source Naturals Skin Eternal Cream
Product Id:  2757
score:  10.385920017584876

Result 7:
Product Title:   Black Opal Dual Complex Fade Gel .9 oz.
Product Id:  307
score:  10.120294266706663

Result 8:
Product Title:   Ambi Fade Cream Normal 2 oz.
Product Id:  116
score:  10.104856807769192

Result 9:
Product Title:   Blistex 

/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [121]:
result_df = searchEngine('Healthy snacks for weight loss', grouped_df, 'review_text_list', result_df)


Result 1:
Product Title:   EAS AdvantEdge Carb Control Nutrition Bar
Product Id:  858
score:  11.461402736621078

Result 2:
Product Title:   SlimFast Shake Mix, 12.83-Ounce (Pack of 3)
Product Id:  2660
score:  10.883786935445341

Result 3:
Product Title:   Bioplex Nutrition Pure WPI Nutritional Shake
Product Id:  296
score:  10.875239248667748

Result 4:
Product Title:   Slim Fast Chocolate Royale Shake Mix Powder, 31.18 oz
Product Id:  2657
score:  10.610792292485053

Result 5:
Product Title:   One-Pound Fat Replica 1Lb Fat Model Replica
Product Id:  2121
score:  10.399986375554263

Result 6:
Product Title:   Natrol Tonalin CLA 1200mg
Product Id:  1803
score:  10.3027280270578

Result 7:
Product Title:   Almased Almased Multi Protein Powder
Product Id:  105
score:  9.779992787186886

Result 8:
Product Title:   ExtendBar, Peanut Butter Chocolate Delight, 1.41-Ounce Bars (Pack of 15)
Product Id:  951
score:  9.27831410112432

Result 9:
Product Title:   Twinlab Ripped Fuel
Product Id: 

/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [122]:
result_df = searchEngine('Natural remedies for joint pain', grouped_df, 'review_text_list', result_df)


Result 1:
Product Title:   Nature's Way Devil's Claw
Product Id:  1870
score:  11.464474960650577

Result 2:
Product Title:   Source Naturals Inflama-Rest
Product Id:  2724
score:  11.39714471104515

Result 3:
Product Title:   H.A. Joint Formula by Purity Products - 90 Capsules [Health and Beauty]
Product Id:  1145
score:  10.77902198514976

Result 4:
Product Title:   Heel Inc Traumeel
Product Id:  1192
score:  10.621527941885237

Result 5:
Product Title:   Doctors Best Best Serrapeptase
Product Id:  753
score:  9.789377235309857

Result 6:
Product Title:   Triflora Arthritis Gel 2.75 Ounces
Product Id:  2986
score:  9.51429060186986

Result 7:
Product Title:   Boericke & Tafel Boericke & Tafel Arniflora Arnica Muscle Pain And Stiffness Gel
Product Id:  330
score:  9.188861312322201

Result 8:
Product Title:   Jarrow Formulas Curcumin 95
Product Id:  1341
score:  9.10236428752617

Result 9:
Product Title:   Boiron Arnica (Arnicare) Gel
Product Id:  333
score:  8.86962451711742

Result

/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_4162382/1506218237.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [123]:
result_df.shape

(1000, 5)

In [126]:
result_df.to_csv('result_df.csv', index = False)

In [2]:
import pandas as pd

In [3]:
result_df_score = pd.read_csv('result_df_score.csv')

In [5]:
result_df_score.head()

,result_index,product_title,product_id,score,query,rel
0,1,"Lansinoh 20265 Disposable Nursing Pads, 60-Co...",1446,6.708870,comfortable clothes,1
1,2,"Seventh Generation Natural Laundry Detergent,...",2622,6.559673,comfortable clothes,5
2,3,Charlie's Soap Laundry Liquid - 1 Gal Jug - (...,503,6.424663,comfortable clothes,5
3,4,Household Essentials Hanging Cotton Canvas La...,1270,6.132265,comfortable clothes,3
4,5,"Wellgate for Women Gel-Comfort Knee Support, ...",3151,6.131872,comfortable clothes,4


In [9]:
result_df_score['map_rel'] = result_df_score['rel']
result_df_score['map_rel'] = result_df_score['map_rel'].apply(lambda x: 1 if x > 3 else 0)

In [10]:
result_df_score

,result_index,product_title,product_id,score,query,rel,map_rel
0,1,"Lansinoh 20265 Disposable Nursing Pads, 60-Co...",1446,6.708870,comfortable clothes,1,0
1,2,"Seventh Generation Natural Laundry Detergent,...",2622,6.559673,comfortable clothes,5,1
2,3,Charlie's Soap Laundry Liquid - 1 Gal Jug - (...,503,6.424663,comfortable clothes,5,1
3,4,Household Essentials Hanging Cotton Canvas La...,1270,6.132265,comfortable clothes,3,0
4,5,"Wellgate for Women Gel-Comfort Knee Support, ...",3151,6.131872,comfortable clothes,4,1
...,...,...,...,...,...,...,...
995,46,Futuro Deluxe Thumb Stabilizer (Pack of 2),1031,6.728875,Natural remedies for joint pain,1,0
996,47,Osteo Bi-Flex Triple Strength - 180 Caplets (...,2181,6.727227,Natural remedies for joint pain,1,0
997,48,"Trigosamine Jointcare Supplement, Maximum Str...",2989,6.725986,Natural remedies for joint pain,1,0
998,49,Ginger Root By Nature's Way 550 Mg,1095,6.716803,Natural remedies for joint pain,1,0


In [94]:
from sklearn.metrics import average_precision_score, ndcg_score

# # Example data
# # Replace these lists with your actual relevance labels and BM25 scores
# y_true = [1, 0, 2, 0, 1]  # Binary relevance labels (1 or 0)
# bm25_scores = [0.9, 0.5, 0.8, 0.2, 0.7]  # BM25 scores

# Calculate MAP using BM25 scores
map_score = average_precision_score(result_df_score['map_rel'], result_df_score['score'])
print("MAP Score:", map_score)

# # Calculate NDCG using BM25 scores (assuming a cutoff value of k=5)
k = 50
ndcg_score = ndcg_score([result_df_score['rel']], [result_df_score['score']], k=k)
print(f"NDCG@{k} Score:", ndcg_score)


MAP Score: 0.6231078222612612
NDCG@50 Score: 0.9201494676370277


In [13]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [14]:
data = ["I love machine learning. Its awesome.",
        "I love coding in python",
        "I love building chatbots",
        "they chat amagingly well"]

tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]

In [15]:
tagged_data

[TaggedDocument(words=['i', 'love', 'machine', 'learning', '.', 'its', 'awesome', '.'], tags=['0']),
 TaggedDocument(words=['i', 'love', 'coding', 'in', 'python'], tags=['1']),
 TaggedDocument(words=['i', 'love', 'building', 'chatbots'], tags=['2']),
 TaggedDocument(words=['they', 'chat', 'amagingly', 'well'], tags=['3'])]

In [23]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# # Example documents
# documents = [
#     "This is the first document.",
#     "This document is the second document.",
#     "And this is the third one.",
#     "Is this the first document?",
# ]

documents = grouped_df['review_text_list']
# Preprocess the documents and create TaggedDocument objects
tagged_data = [TaggedDocument(words=document, tags=[str(i)]) for i, document in enumerate(documents)]

In [25]:
# Train a Doc2Vec model
model = Doc2Vec(vector_size=50, min_count=2, epochs=20)
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

In [32]:
grouped_df.head()

,product/title,review_text_list,productId
0,#1 Rated Diaper Happy Heiny's Pocket Diaper w...,"[deliver, promised, easy, use, wash, leaking, ...",1
1,(Pack of 3) Reach Access Power Flosser (Color...,"[cannot, help, assert, reach, access, daily, f...",2
2,1 Step-Compac 14102 1 Step Tub Shower Cleaner...,"[love, little, pads, work, well, shower, alway...",3
3,100 Durex Maximum Love Condoms NEW! Larger an...,"[purchased, company, green, web, properties, c...",4
4,100 Wilkinson Sword Classic Double Edge Safet...,"[order, went, flawlessly, delivered, within, t...",5


In [56]:
# Infer vector for a new document
new_document = "vitamin"
inferred_vector = model.infer_vector(new_document.lower().split())

top_n = 5
similar_documents = model.dv.most_similar([inferred_vector], topn=top_n)

# Print ranked documents based on similarity
print("Top 5 Documents:")
for rank, (index, score) in enumerate(similar_documents, start=1):
    print(f"Rank {rank}: Document {index}, Similarity Score {score}")
    print(grouped_df.loc[grouped_df['productId'] == int(index)+1, 'product/title'])
    print(grouped_df['product/title'][int(index)])
    print()
    result_df_1 = result_df_1.append({
        'result_index': rank,
        'product_title': dataframe['product/title'][int(index)],
        'product_id': dataframe['productId'][int(index)],
        'score': score,
        'query': input_query
    }, ignore_index=True)

    

Top 5 Documents:
Rank 1: Document 2463, Similarity Score 0.8006784319877625
2463     Rainbow Light Complete B Complex
Name: product/title, dtype: object
 Rainbow Light Complete B Complex

Rank 2: Document 2115, Similarity Score 0.8004937171936035
2115     One Daily Multi
Name: product/title, dtype: object
 One Daily Multi

Rank 3: Document 488, Similarity Score 0.787991464138031
488     Centrum Multivitamin - Adults Under 50
Name: product/title, dtype: object
 Centrum Multivitamin - Adults Under 50

Rank 4: Document 2732, Similarity Score 0.7814715504646301
2732     Source Naturals Life Force Multiple, No Iron ...
Name: product/title, dtype: object
 Source Naturals Life Force Multiple, No Iron Tablets

Rank 5: Document 1188, Similarity Score 0.7809725403785706
1188     Heaven Sent Sea Essentials
Name: product/title, dtype: object
 Heaven Sent Sea Essentials



In [ ]:
def doc2vecEngine(input_query, dataframe = grouped_df ):
    documents = dataframe['review_text_list']
    tagged_data = [TaggedDocument(words=document, tags=[str(i)]) for i, document in enumerate(documents)]
    model = Doc2Vec(vector_size=50, min_count=2, epochs=20)
    model.build_vocab(tagged_data)
    model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)
    
    
    inferred_vector = model.infer_vector(input_query.lower().split())

    top_n = 5
    similar_documents = model.dv.most_similar([inferred_vector], topn=top_n)

    # Print ranked documents based on similarity
    print("Top 5 Documents:")
    for rank, (index, score) in enumerate(similar_documents, start=1):
        print(f"Rank {rank}: Document {index}, Similarity Score {score}")
        print(dataframe['product/title'][int(index)])
        print()
        result_df_1 = result_df_1.append({
            'result_index': rank,
            'product_title': dataframe['product/title'][int(index)],
            'product_id': dataframe['productId'][int(index)],
            'score': score,
            'query': input_query
        }, ignore_index=True)

In [65]:
## real
def doc2vecEngine(input_query, dataframe, result_df, top_n):
    tagged_data = [TaggedDocument(words=words, tags=[str(i)]) for i, words in enumerate(dataframe['review_text_list'])]

    model = Doc2Vec(vector_size=50, min_count=2, epochs=20)
    model.build_vocab(tagged_data)
    model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)
    
    inferred_vector = model.infer_vector(input_query.lower().split())

    similar_documents = model.dv.most_similar([inferred_vector], topn=top_n)

    # Print ranked documents based on similarity
    print("Top 50 Documents:")
    for rank, (index, score) in enumerate(similar_documents, start=1):
        print(f"Rank {rank}: Document {index}, Similarity Score {score}")
        print(dataframe['product/title'].iloc[int(index)])
        print()
        
        # Convert indices to integers
        result_df = result_df.append({
            'result_index': rank,
            'product_title': dataframe['product/title'].iloc[int(index)],
            'product_id': dataframe['productId'].iloc[int(index)],
            'score': score,
            'query': input_query
        }, ignore_index=True)
    
    return result_df

In [64]:
result_df_1 = pd.DataFrame(columns=['result_index', 'product_title', 'product_id', 'score', 'query'])
top_n = 50

In [66]:
result_df_1 = doc2vecEngine('comfortable clothes', grouped_df, result_df_1, top_n)

Top 50 Documents:
Rank 1: Document 318, Similarity Score 0.6609342098236084
 Blueberry Diaper Bag

Rank 2: Document 2623, Similarity Score 0.6110565662384033
 Seventh Generation Natural Laundry Detergent, Lavender Scented, Ultra-Concentrated Liquid, 100-Ounce Bottles (Pack of 4)

Rank 3: Document 3077, Similarity Score 0.5965548753738403
 Vinyl Exam Gloves. Powder Free.

Rank 4: Document 2619, Similarity Score 0.5961759686470032
 Seventh Generation Natural Fabric Softener, Blue Eucalyptus and Lavender Scent, Ultra-Concentrated, 40-Ounce Bottles (Pack of 6)

Rank 5: Document 1371, Similarity Score 0.5956820845603943
 Jobst SupportWear Ultra Sheer Knee High

Rank 6: Document 73, Similarity Score 0.5893173813819885
 Activa Knee High Light Compression Sheer 15-20 mm Hg

Rank 7: Document 958, Similarity Score 0.5813047289848328
 FLA Safe-T-Sport Sports Neoprene Knee Support Brace

Rank 8: Document 1720, Similarity Score 0.5759525895118713
 Mrs. Meyer's Clean Day Fabric Softener, 32-Ounce Bo

/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [67]:
result_df_1 = doc2vecEngine('warm clothes', grouped_df, result_df_1, top_n)

Top 50 Documents:
Rank 1: Document 2917, Similarity Score 0.6129825115203857
 The Original Tube Reader

Rank 2: Document 318, Similarity Score 0.6121980547904968
 Blueberry Diaper Bag

Rank 3: Document 2056, Similarity Score 0.6068845391273499
 OTC Cock-Up Wrist Splint

Rank 4: Document 1342, Similarity Score 0.5954672694206238
 Jarrow Formulas Evening Primrose Oil

Rank 5: Document 2960, Similarity Score 0.583687424659729
 Tide HE Detergent for High Efficiency Washers, Powder, Original Scent, 31-Load Boxes (Pack of 4)

Rank 6: Document 64, Similarity Score 0.5785584449768066
 ADC ADVANTAGE Wrist Blood Pressure Monitor

Rank 7: Document 98, Similarity Score 0.578162431716919
 All Small & Mighty, Triple Concentrated Liquid Laundry Detergent, Nine 32-Load Bottles (288-Loads)

Rank 8: Document 3189, Similarity Score 0.5724849104881287
 Zafu Yoga Sitting Meditation Cushion - Buckwheat Filling

Rank 9: Document 2494, Similarity Score 0.5714332461357117
 Relora 250 mg 90 Caps

Rank 10: Docum

/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [68]:
result_df_1 = doc2vecEngine('travel', grouped_df, result_df_1, top_n)

Top 50 Documents:
Rank 1: Document 3189, Similarity Score 0.6627400517463684
 Zafu Yoga Sitting Meditation Cushion - Buckwheat Filling

Rank 2: Document 1176, Similarity Score 0.649790346622467
 Health Plus Colon Cleanse

Rank 3: Document 1362, Similarity Score 0.6348022222518921
 JetLine Z-Torch Dual Flame Torch lighter

Rank 4: Document 835, Similarity Score 0.634111762046814
 Duro-Med Portable Bidet/Sitz Bath

Rank 5: Document 1006, Similarity Score 0.6255248785018921
 Floxite 5X Lighted Magnifying Mirror 5" white

Rank 6: Document 2463, Similarity Score 0.624592125415802
 Rainbow Light Complete B Complex

Rank 7: Document 3041, Similarity Score 0.6198139190673828
 Ultraviolet Set Eau de Parfum for Women by Paco Rabanne

Rank 8: Document 1910, Similarity Score 0.6164989471435547
 Nature's Way Vitamin C 1000 with Rose Hips

Rank 9: Document 1889, Similarity Score 0.6159552931785583
 Nature's Way Magnesium

Rank 10: Document 3029, Similarity Score 0.6149962544441223
 ULTRA SOURCE OF L

/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [69]:
result_df_1 = doc2vecEngine('conditioner', grouped_df, result_df_1, top_n)

Top 50 Documents:
Rank 1: Document 182, Similarity Score 0.7590571641921997
 Avalon Organic Conditioner

Rank 2: Document 183, Similarity Score 0.7467320561408997
 Avalon Organics Conditioner Awapuhi Mango Moisturizing

Rank 3: Document 184, Similarity Score 0.7435572147369385
 Avalon Organics Conditioner Strenthening Peppermint -- 11 fl oz

Rank 4: Document 1096, Similarity Score 0.7294250130653381
 Giovanni Cosmetics Conditioner Smooth As Silk

Rank 5: Document 185, Similarity Score 0.7196840643882751
 Avalon Shampoo

Rank 6: Document 2460, Similarity Score 0.7128773927688599
 RPS 16-Ounce Waterbed Conditioner

Rank 7: Document 861, Similarity Score 0.6953190565109253
 EO Products Shampoo

Rank 8: Document 1740, Similarity Score 0.6778215169906616
 Mustela Dermo-cleansing

Rank 9: Document 1739, Similarity Score 0.6765319108963013
 Mustela 2 in 1 Hair & Body Shampoo

Rank 10: Document 88, Similarity Score 0.6673492789268494
 Africa's Best Ultimate Herbal Oil 8 oz.

Rank 11: Document 

/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [70]:
result_df_1 = doc2vecEngine('organic', grouped_df, result_df_1, top_n)

Top 50 Documents:
Rank 1: Document 2585, Similarity Score 0.6604780554771423
 Sambazon Organic Acai Powder

Rank 2: Document 118, Similarity Score 0.652195155620575
 American Baby Company Cotton Terry Flat Fitted Changing Pad Cover

Rank 3: Document 1816, Similarity Score 0.6416242122650146
 Natural Selection Bath and Body's 1/3 oz Essential Oil

Rank 4: Document 276, Similarity Score 0.6403576731681824
 Berry Green 360 gram - New Chapter ( Fast Shipping )

Rank 5: Document 852, Similarity Score 0.636508047580719
 Dynamic Health Laboratories Organic Apple Cider Vinegar with Mother

Rank 6: Document 393, Similarity Score 0.6358325481414795
 Brewer's Yeast 16 oz (454 g) Pwdr

Rank 7: Document 1338, Similarity Score 0.6352764368057251
 Jarrow Formulas Brown Rice Protein Chocolate Flavor

Rank 8: Document 1022, Similarity Score 0.629185140132904
 Fresh Bee Pollen Whole Granules - 16 oz. - Granules

Rank 9: Document 2962, Similarity Score 0.6281954050064087
 Tide Powder Detergent, Original 

/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [71]:
result_df_1 = doc2vecEngine('healthy', grouped_df, result_df_1, top_n)

Top 50 Documents:
Rank 1: Document 1114, Similarity Score 0.6978934407234192
 Golden Alaska Deep Sea Omega-3-6-9 Fish Oil 1000mg

Rank 2: Document 1805, Similarity Score 0.695030689239502
 Natural Balance Ultra Colon Clenz

Rank 3: Document 238, Similarity Score 0.691184401512146
 Balanceuticals Seabuckthorn Seed Oil, 500 mg Dietary Supplement Softgels, 60-Count Bottle

Rank 4: Document 2766, Similarity Score 0.6739925146102905
 Source Naturals Turmeric Extract

Rank 5: Document 2463, Similarity Score 0.6714263558387756
 Rainbow Light Complete B Complex

Rank 6: Document 2009, Similarity Score 0.6654887795448303
 Now Foods, L-Carnosine 500 mg

Rank 7: Document 1421, Similarity Score 0.6651158928871155
 Kyolic Garlic Formula 106 Circulation

Rank 8: Document 2733, Similarity Score 0.6606178283691406
 Source Naturals Life Force Multiple, No Iron, 120 Tablets

Rank 9: Document 2022, Similarity Score 0.6529693007469177
 Now Foods, Vitamin B-50 tablets

Rank 10: Document 3189, Similarity Sc

/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [72]:
result_df_1

,result_index,product_title,product_id,score,query
0,1,Blueberry Diaper Bag,319,0.660934,comfortable clothes
1,2,"Seventh Generation Natural Laundry Detergent,...",2624,0.611057,comfortable clothes
2,3,Vinyl Exam Gloves. Powder Free.,3078,0.596555,comfortable clothes
3,4,"Seventh Generation Natural Fabric Softener, B...",2620,0.596176,comfortable clothes
4,5,Jobst SupportWear Ultra Sheer Knee High,1372,0.595682,comfortable clothes
...,...,...,...,...,...
295,46,Fresh Bee Pollen Whole Granules - 16 oz. - Gr...,1023,0.602725,healthy
296,47,Mega Green Tea Extract (Decaffeinated),1638,0.602594,healthy
297,48,Source Naturals Women's Life Force Multiple,2774,0.600229,healthy
298,49,"Green Foods Green Magma, 2.8 Ounce",1127,0.600227,healthy


In [73]:
result_df_1 = doc2vecEngine('weight control', grouped_df, result_df_1, top_n)

Top 50 Documents:
Rank 1: Document 3189, Similarity Score 0.793338418006897
 Zafu Yoga Sitting Meditation Cushion - Buckwheat Filling

Rank 2: Document 1522, Similarity Score 0.7195961475372314
 Love Liquid 100ml Water Base D Lube

Rank 3: Document 1816, Similarity Score 0.6912747025489807
 Natural Selection Bath and Body's 1/3 oz Essential Oil

Rank 4: Document 2766, Similarity Score 0.681230366230011
 Source Naturals Turmeric Extract

Rank 5: Document 1390, Similarity Score 0.6709662675857544
 K-Y K-Y Personal Lubricant Jelly

Rank 6: Document 1299, Similarity Score 0.660801887512207
 Id Lubes Millennium

Rank 7: Document 835, Similarity Score 0.6534042358398438
 Duro-Med Portable Bidet/Sitz Bath

Rank 8: Document 1805, Similarity Score 0.6519701480865479
 Natural Balance Ultra Colon Clenz

Rank 9: Document 1818, Similarity Score 0.6511913537979126
 Natural Selection Bath and Body's Zen Massage Oil 8 oz

Rank 10: Document 2309, Similarity Score 0.6486340761184692
 Philips Norelco HQ4

/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [74]:
result_df_1 = doc2vecEngine('nutrition', grouped_df, result_df_1, top_n)

Top 50 Documents:
Rank 1: Document 917, Similarity Score 0.7807632684707642
 Ensure Complete Balanced Nutrition Drink, Butter Pecan, 8-Ounce (Pack of 24)

Rank 2: Document 276, Similarity Score 0.7456840872764587
 Berry Green 360 gram - New Chapter ( Fast Shipping )

Rank 3: Document 2463, Similarity Score 0.7168024182319641
 Rainbow Light Complete B Complex

Rank 4: Document 1188, Similarity Score 0.7156192660331726
 Heaven Sent Sea Essentials

Rank 5: Document 1357, Similarity Score 0.7119084000587463
 Jay Robb Enterprises - Whey Vanilla Isolate, 12 oz powder

Rank 6: Document 2710, Similarity Score 0.7081761360168457
 Source Naturals Chlorella from Yaeyama Powder

Rank 7: Document 489, Similarity Score 0.7050666809082031
 Centrum Performance Multivitamin/Multimineral Supplement, Tablets

Rank 8: Document 1328, Similarity Score 0.7033829689025879
 IsoFlex - Whey Protein Isolate

Rank 9: Document 757, Similarity Score 0.7005708813667297
 Douglas Labs - Ultra Preventive X 240 Tabs [Hea

/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [75]:
result_df_1 = doc2vecEngine('energy bar', grouped_df, result_df_1, top_n)

Top 50 Documents:
Rank 1: Document 3189, Similarity Score 0.758764922618866
 Zafu Yoga Sitting Meditation Cushion - Buckwheat Filling

Rank 2: Document 1910, Similarity Score 0.700538158416748
 Nature's Way Vitamin C 1000 with Rose Hips

Rank 3: Document 1537, Similarity Score 0.6613627672195435
 MB1017 Reading Glasses w/Spring Hinge and Subtle Stylish Stripe

Rank 4: Document 1805, Similarity Score 0.6546276807785034
 Natural Balance Ultra Colon Clenz

Rank 5: Document 2309, Similarity Score 0.6527072787284851
 Philips Norelco HQ4 Micro Action Plus Lift-and-Cut Tripleheader Replacement Heads

Rank 6: Document 2961, Similarity Score 0.6498165130615234
 Tide HE Original Scent with Actilift

Rank 7: Document 944, Similarity Score 0.6462002992630005
 Eveready Energizer Alkaline Battery Pack - Alkaline - 595mAh - 9V DC

Rank 8: Document 2766, Similarity Score 0.6440891623497009
 Source Naturals Turmeric Extract

Rank 9: Document 1522, Similarity Score 0.6420097351074219
 Love Liquid 100ml 

/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [76]:
result_df_1 = doc2vecEngine('Best vitamins for immune system', grouped_df, result_df_1, top_n)

Top 50 Documents:
Rank 1: Document 2119, Similarity Score 0.7712838053703308
 One-A-Day Essential High Potency Multivitamin -- 130 Tablets

Rank 2: Document 488, Similarity Score 0.7337431311607361
 Centrum Multivitamin - Adults Under 50

Rank 3: Document 2466, Similarity Score 0.7140817642211914
 Rainbow Light Dha Smart Essentials

Rank 4: Document 2136, Similarity Score 0.7096953988075256
 Optimum Nutrition Opti-Women, Women's Multivitamin

Rank 5: Document 490, Similarity Score 0.699475884437561
 Centrum Silver - Adults 50+

Rank 6: Document 2115, Similarity Score 0.6956958770751953
 One Daily Multi

Rank 7: Document 1911, Similarity Score 0.6863002777099609
 Nature's Way Vitamin C 500

Rank 8: Document 1188, Similarity Score 0.6844677925109863
 Heaven Sent Sea Essentials

Rank 9: Document 2117, Similarity Score 0.6812083125114441
 One a Day women's multi vitamin 250 tablets

Rank 10: Document 1636, Similarity Score 0.6808258891105652
 Mega Food - Women's One Daily

Rank 11: Documen

/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [77]:
result_df_1 = doc2vecEngine('Natural remedies for sleep', grouped_df, result_df_1, top_n)

Top 50 Documents:
Rank 1: Document 2737, Similarity Score 0.7306613326072693
 Source Naturals Melatonin 2.5mg, Orange

Rank 2: Document 2065, Similarity Score 0.6986934542655945
 OTC Soft-Fit Suede Finish Wrist Brace

Rank 3: Document 2739, Similarity Score 0.6777198910713196
 Source Naturals Melatonin 3mg Vegetarian Capsules

Rank 4: Document 2745, Similarity Score 0.6753210425376892
 Source Naturals NutraSleep

Rank 5: Document 1908, Similarity Score 0.6748923063278198
 Nature's Way Valerian Standardized Extract

Rank 6: Document 823, Similarity Score 0.6427437663078308
 Duro-Med Elevating Leg Rest with Blue Polyester/Cotton Cover

Rank 7: Document 403, Similarity Score 0.6423916220664978
 Bucky Eye Shades

Rank 8: Document 2643, Similarity Score 0.6388258337974548
 Simmons Beautyrest Sleep Massager

Rank 9: Document 839, Similarity Score 0.6375019550323486
 Duro-Med Stress-Ease Pillow Support, White

Rank 10: Document 1489, Similarity Score 0.6341326236724854
 Lights Out! Sleep Mask

/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [78]:
result_df_1 = doc2vecEngine('Top-rated fitness trackers', grouped_df, result_df_1, top_n)

Top 50 Documents:
Rank 1: Document 2009, Similarity Score 0.7062566876411438
 Now Foods, L-Carnosine 500 mg

Rank 2: Document 2796, Similarity Score 0.6297537684440613
 Stacker 2 Ephedra Free

Rank 3: Document 2765, Similarity Score 0.6239801645278931
 Source Naturals Tonalin 1000 CLA

Rank 4: Document 1573, Similarity Score 0.612220048904419
 Mark of Fitness MF-83 Wrist Blood Pressure Monitor

Rank 5: Document 757, Similarity Score 0.6042536497116089
 Douglas Labs - Ultra Preventive X 240 Tabs [Health and Beauty]

Rank 6: Document 2766, Similarity Score 0.5954965353012085
 Source Naturals Turmeric Extract

Rank 7: Document 2056, Similarity Score 0.5917456150054932
 OTC Cock-Up Wrist Splint

Rank 8: Document 2725, Similarity Score 0.5915435552597046
 Source Naturals L-Arginine L-Citrulline Complex

Rank 9: Document 1926, Similarity Score 0.5909085869789124
 Neurovar (30 Capsules) Increase Your Memory, Focus & Mental Clarity!

Rank 10: Document 1973, Similarity Score 0.5894327163696289


/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [79]:
result_df_1 = doc2vecEngine('Organic protein powders', grouped_df, result_df_1, top_n)

Top 50 Documents:
Rank 1: Document 3189, Similarity Score 0.7150318622589111
 Zafu Yoga Sitting Meditation Cushion - Buckwheat Filling

Rank 2: Document 1522, Similarity Score 0.6821980476379395
 Love Liquid 100ml Water Base D Lube

Rank 3: Document 1816, Similarity Score 0.6754084229469299
 Natural Selection Bath and Body's 1/3 oz Essential Oil

Rank 4: Document 1384, Similarity Score 0.6731236577033997
 Johnson's Foot Soap, Economy Size, 8 packets [8 oz (226 g)], (Pack of 4)

Rank 5: Document 1818, Similarity Score 0.6630471348762512
 Natural Selection Bath and Body's Zen Massage Oil 8 oz

Rank 6: Document 1593, Similarity Score 0.6535503268241882
 Master Massage Fleece Pad

Rank 7: Document 2961, Similarity Score 0.6481483578681946
 Tide HE Original Scent with Actilift

Rank 8: Document 835, Similarity Score 0.622145414352417
 Duro-Med Portable Bidet/Sitz Bath

Rank 9: Document 2621, Similarity Score 0.6163874268531799
 Seventh Generation Natural Laundry Detergent, Baby, Ultra-Conce

/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [80]:
result_df_1 = doc2vecEngine('Effective allergy relief medications', grouped_df, result_df_1, top_n)

Top 50 Documents:
Rank 1: Document 3189, Similarity Score 0.76882004737854
 Zafu Yoga Sitting Meditation Cushion - Buckwheat Filling

Rank 2: Document 1522, Similarity Score 0.7173349261283875
 Love Liquid 100ml Water Base D Lube

Rank 3: Document 1816, Similarity Score 0.6922295093536377
 Natural Selection Bath and Body's 1/3 oz Essential Oil

Rank 4: Document 911, Similarity Score 0.6900449395179749
 Energizer Lithium Coin Blister Pack Watch/Electronic Batteries, 2 - Count (Pack of 12)

Rank 5: Document 2309, Similarity Score 0.6860761642456055
 Philips Norelco HQ4 Micro Action Plus Lift-and-Cut Tripleheader Replacement Heads

Rank 6: Document 1537, Similarity Score 0.6833785176277161
 MB1017 Reading Glasses w/Spring Hinge and Subtle Stylish Stripe

Rank 7: Document 1377, Similarity Score 0.6817771196365356
 Jobst Women's UltraSheer Firm Support Knee Highs

Rank 8: Document 2056, Similarity Score 0.6810483336448669
 OTC Cock-Up Wrist Splint

Rank 9: Document 2357, Similarity Score 0.

/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [81]:
result_df_1 = doc2vecEngine('release anxiety', grouped_df, result_df_1, top_n)

Top 50 Documents:
Rank 1: Document 2494, Similarity Score 0.725533664226532
 Relora 250 mg 90 Caps

Rank 2: Document 2737, Similarity Score 0.6806674599647522
 Source Naturals Melatonin 2.5mg, Orange

Rank 3: Document 1892, Similarity Score 0.6732202172279358
 Nature's Way Melissa/Lemon Balm

Rank 4: Document 1799, Similarity Score 0.6696153283119202
 Natrol (incl Laci Le Beau Teas) Natrol Melatonin

Rank 5: Document 2727, Similarity Score 0.6689903736114502
 Source Naturals L-Theanine 200mg

Rank 6: Document 44, Similarity Score 0.6680504083633423
 5-HTP 200 mg - 60 Vcaps

Rank 7: Document 1757, Similarity Score 0.6673691868782043
 NOW Food Melatonin 3 mg 60 Capsules

Rank 8: Document 1908, Similarity Score 0.6666579842567444
 Nature's Way Valerian Standardized Extract

Rank 9: Document 2739, Similarity Score 0.6657427549362183
 Source Naturals Melatonin 3mg Vegetarian Capsules

Rank 10: Document 1977, Similarity Score 0.6534849405288696
 Now Foods Holy Basil Extract

Rank 11: Documen

/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [82]:
result_df_1 = doc2vecEngine('Recommended probiotics for gut health', grouped_df, result_df_1, top_n)

Top 50 Documents:
Rank 1: Document 3189, Similarity Score 0.7422338724136353
 Zafu Yoga Sitting Meditation Cushion - Buckwheat Filling

Rank 2: Document 1522, Similarity Score 0.7146081328392029
 Love Liquid 100ml Water Base D Lube

Rank 3: Document 1805, Similarity Score 0.6725359559059143
 Natural Balance Ultra Colon Clenz

Rank 4: Document 835, Similarity Score 0.6704562306404114
 Duro-Med Portable Bidet/Sitz Bath

Rank 5: Document 2766, Similarity Score 0.651880145072937
 Source Naturals Turmeric Extract

Rank 6: Document 911, Similarity Score 0.6401391625404358
 Energizer Lithium Coin Blister Pack Watch/Electronic Batteries, 2 - Count (Pack of 12)

Rank 7: Document 1910, Similarity Score 0.6392278075218201
 Nature's Way Vitamin C 1000 with Rose Hips

Rank 8: Document 1421, Similarity Score 0.6345491409301758
 Kyolic Garlic Formula 106 Circulation

Rank 9: Document 2056, Similarity Score 0.6331347227096558
 OTC Cock-Up Wrist Splint

Rank 10: Document 1816, Similarity Score 0.632105

/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [83]:
result_df_1 = doc2vecEngine('Essential oils for stress relief', grouped_df, result_df_1, top_n)

Top 50 Documents:
Rank 1: Document 734, Similarity Score 0.7187196612358093
 Doans extra strength pain reliever caplets - 24 ea

Rank 2: Document 2036, Similarity Score 0.6823177933692932
 Nutri-West - DSF FORMULA - 120

Rank 3: Document 2439, Similarity Score 0.6315901279449463
 Prossage Heat

Rank 4: Document 3189, Similarity Score 0.6276036500930786
 Zafu Yoga Sitting Meditation Cushion - Buckwheat Filling

Rank 5: Document 1807, Similarity Score 0.6239491701126099
 Natural Care Vein-Gard

Rank 6: Document 3200, Similarity Score 0.619411826133728
 Zheng Gu Shui External Analgesic Lotion - 3.4 Fl. Oz. (100ml)

Rank 7: Document 1111, Similarity Score 0.6098048090934753
 Glucosamine Plus MSM

Rank 8: Document 2494, Similarity Score 0.5968154072761536
 Relora 250 mg 90 Caps

Rank 9: Document 2949, Similarity Score 0.5931503772735596
 Thermoskin Wrist Brace, Hand Brace, Carpal Tunnel Brace, Beige, Left

Rank 10: Document 959, Similarity Score 0.5911020040512085
 FLEXCIN W/CM8 JOINT HEALT

/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [84]:
result_df_1 = doc2vecEngine('Top-rated sunscreen for sensitive skin', grouped_df, result_df_1, top_n)

Top 50 Documents:
Rank 1: Document 911, Similarity Score 0.7056635618209839
 Energizer Lithium Coin Blister Pack Watch/Electronic Batteries, 2 - Count (Pack of 12)

Rank 2: Document 3189, Similarity Score 0.6985123157501221
 Zafu Yoga Sitting Meditation Cushion - Buckwheat Filling

Rank 3: Document 1537, Similarity Score 0.6775862574577332
 MB1017 Reading Glasses w/Spring Hinge and Subtle Stylish Stripe

Rank 4: Document 1816, Similarity Score 0.6744695901870728
 Natural Selection Bath and Body's 1/3 oz Essential Oil

Rank 5: Document 45, Similarity Score 0.6709079146385193
 50 392 LR41 Watch Alkaline Batteries AG3 SR41 Battery

Rank 6: Document 2309, Similarity Score 0.6661174297332764
 Philips Norelco HQ4 Micro Action Plus Lift-and-Cut Tripleheader Replacement Heads

Rank 7: Document 944, Similarity Score 0.665654718875885
 Eveready Energizer Alkaline Battery Pack - Alkaline - 595mAh - 9V DC

Rank 8: Document 1377, Similarity Score 0.665493369102478
 Jobst Women's UltraSheer Firm Sup

/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [85]:
result_df_1 = doc2vecEngine('Healthy snacks for weight loss', grouped_df, result_df_1, top_n)

Top 50 Documents:
Rank 1: Document 911, Similarity Score 0.6741309762001038
 Energizer Lithium Coin Blister Pack Watch/Electronic Batteries, 2 - Count (Pack of 12)

Rank 2: Document 2961, Similarity Score 0.6580809950828552
 Tide HE Original Scent with Actilift

Rank 3: Document 2056, Similarity Score 0.6577515602111816
 OTC Cock-Up Wrist Splint

Rank 4: Document 1411, Similarity Score 0.6528103351593018
 Kleenite denture cleanser powder - 9 oz

Rank 5: Document 1378, Similarity Score 0.6494649052619934
 Jobst Women's UltraSheer Firm Support Pantyhose

Rank 6: Document 3189, Similarity Score 0.6481884121894836
 Zafu Yoga Sitting Meditation Cushion - Buckwheat Filling

Rank 7: Document 1474, Similarity Score 0.6391131281852722
 LifeSavers Sugar Free Wint-O-Green Hard Candy, 12 - 2.75-Ounce Bags

Rank 8: Document 944, Similarity Score 0.6355255246162415
 Eveready Energizer Alkaline Battery Pack - Alkaline - 595mAh - 9V DC

Rank 9: Document 561, Similarity Score 0.6289228200912476
 Clorox

/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [86]:
result_df_1 = doc2vecEngine('Natural remedies for joint pain', grouped_df, result_df_1, top_n)

Top 50 Documents:
Rank 1: Document 3189, Similarity Score 0.7474644780158997
 Zafu Yoga Sitting Meditation Cushion - Buckwheat Filling

Rank 2: Document 1816, Similarity Score 0.6994559168815613
 Natural Selection Bath and Body's 1/3 oz Essential Oil

Rank 3: Document 2494, Similarity Score 0.6749046444892883
 Relora 250 mg 90 Caps

Rank 4: Document 1537, Similarity Score 0.660709023475647
 MB1017 Reading Glasses w/Spring Hinge and Subtle Stylish Stripe

Rank 5: Document 2961, Similarity Score 0.6439895033836365
 Tide HE Original Scent with Actilift

Rank 6: Document 1528, Similarity Score 0.6419991850852966
 Lynco Sports 400 Series - L400

Rank 7: Document 2309, Similarity Score 0.6344348788261414
 Philips Norelco HQ4 Micro Action Plus Lift-and-Cut Tripleheader Replacement Heads

Rank 8: Document 756, Similarity Score 0.6239486932754517
 Dona Crystalline Glucosamine Sulfate Caplets - 60 Ea

Rank 9: Document 1377, Similarity Score 0.6228672862052917
 Jobst Women's UltraSheer Firm Suppo

/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
/tmp/ipykernel_2456364/2887253033.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [88]:
result_df_1.to_csv('result_df_1.csv', index = False)

In [97]:
result_df_score_1 = pd.read_csv('result_df_score_1.csv')

In [98]:
result_df_score_1['map_rel'] = result_df_score_1['rel']
result_df_score_1['map_rel'] = result_df_score_1['map_rel'].apply(lambda x: 1 if x > 3 else 0)

In [99]:
from sklearn.metrics import average_precision_score, ndcg_score

# # Example data
# # Replace these lists with your actual relevance labels and BM25 scores
# y_true = [1, 0, 2, 0, 1]  # Binary relevance labels (1 or 0)
# bm25_scores = [0.9, 0.5, 0.8, 0.2, 0.7]  # BM25 scores

# Calculate MAP using BM25 scores
map_score_1 = average_precision_score(result_df_score_1['map_rel'], result_df_score_1['score'])
print("MAP Score:", map_score_1)

# # Calculate NDCG using BM25 scores (assuming a cutoff value of k=5)
k = 50
ndcg_score_1 = ndcg_score([result_df_score_1['rel']], [result_df_score_1['score']], k=k)
print(f"NDCG@{k} Score:", ndcg_score_1)

MAP Score: 0.6641798893349979
NDCG@50 Score: 0.9260201684625474
